In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [46]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 

import numpy as np
import pandas as pd

In [4]:
tokenizer = Tokenizer()

#get the data
data = open('../input/mythesis-txt/mythesis.txt').read()
corpus = data.lower().replace('\n','. ').split('. ')

tokenizer.fit_on_texts(corpus)

In [5]:
#creating the word index 
total_words = len(tokenizer.word_index)+1
total_words

In [6]:
#create the input sequences
input_sequences= []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [7]:
#pad the input sequences
max_sequences_len = max([len(x) for x in input_sequences])
print(max_sequences_len)

padded_input_sequences = np.array(pad_sequences(input_sequences, 
                                       maxlen= max_sequences_len,
                                       padding='pre'))

print(padded_input_sequences[:5])

In [8]:
#create predictors and label
predictors, label = padded_input_sequences[:,:-1],padded_input_sequences[:,-1]
label = ku.to_categorical(label, num_classes=total_words)

In [9]:
predictors

In [24]:
#the model 
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequences_len-1),
    tf.keras.layers.Bidirectional(LSTM(200,return_sequences=True)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(LSTM(64)),
    tf.keras.layers.Dense(1024,activation='relu',kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dense(total_words, activation = 'softmax')
])

optimizer ='Adam'

model.compile(loss = 'categorical_crossentropy', 
              optimizer = optimizer, 
             metrics = ['acc'])
model.summary()

In [25]:
 history = model.fit(predictors,
                     label, 
                     epochs=100, 
                     verbose=1)

In [43]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.DataFrame(history.history)

In [44]:
df.plot(figsize=(8,5))
plt.xlabel('epochs')
plt.title/

In [47]:
seed_text = 'Drug discovery for G-protein-coupled receptors'
next_words = 100

for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],maxlen = max_sequences_len-1, padding = 'pre')
        predicted = np.argmax(model.predict(token_list, verbose=0),axis = -1)
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word 
                break
        seed_text += ' ' + output_word
print(seed_text)